<a href="https://colab.research.google.com/github/rrfsantos/Airbnb-Segmentacao-dos-principais-assuntos-das-reviews/blob/main/1_Airbnb_pre_processamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install contractions

In [ ]:
!pip install googletrans==3.1.0a0

In [ ]:
# Importando bibliotecas
import pandas as pd
import numpy as np
import warnings
import nltk
#import spacy
import re
import string
#from contractions import contractions_dict
from googletrans import Translator
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

warnings.filterwarnings('ignore')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
np.random.seed(0)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
#!python -m spacy download en_core_web_lg
#!python -m spacy link en_core_web_lg en --force

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
workdir_path = '/content/drive/My Drive/Airbnb/'  # Inserir o local da pasta onde estão os arquivos de entrada (treino e teste)
os.chdir(workdir_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Análise exploratória dos dados

In [ ]:
# Carregando dataset reviews
df_full = pd.read_csv('reviews.csv', engine='python')

In [ ]:
# Verificando as dimensões do dataset completo
print('------- Dimensões do dataset completo --------')
print("Variáveis:\t{}\nEntradas:\t{}\n".format(df_full.shape[1], df_full.shape[0]))

------- Dimensões do dataset completo --------
Variáveis:	6
Entradas:	356118



In [ ]:
# Selecionando 30% do dataset para construção do modelo
df = df_full.sample(frac=0.30)

In [ ]:
# Verificando as dimensões do dataset "sample"
print('------- Dimensões do dataset --------')
print("Variáveis:\t{}\nEntradas:\t{}\n".format(df.shape[1], df.shape[0]))

------- Dimensões do dataset --------
Variáveis:	6
Entradas:	3561



In [ ]:
# Identificando os tipos de dados
df.dtypes

listing_id        int64
id                int64
date             object
reviewer_id       int64
reviewer_name    object
comments         object
dtype: object

In [ ]:
# Exemplos de registros do dataset
df.head(2)

,listing_id,id,date,reviewer_id,reviewer_name,comments
315919,40652300,710542759,2020-11-22,20863004,Teresa,"O Apartamento é muitíssimo bem localizado, com..."
237975,21964073,751957974,2021-04-25,25503568,Cynara,A estadia fói ótima. A casa possui boa localiz...


In [ ]:
# Verificando valores nulos
df.isna().sum()

listing_id       0
id               0
date             0
reviewer_id      0
reviewer_name    0
comments         1
dtype: int64

#### Pré-processamento do texto



In [ ]:
# Removendo linhas com missing values
df = df.dropna()

In [ ]:
# Verificando as novas dimensões dataset
print("Variáveis:\t{}\nEntradas:\t{}\n".format(df.shape[1], df.shape[0]))

Variáveis:	6
Entradas:	3560



In [ ]:
display(df['comments'][315919])
print('\n')
display(df['comments'][237975])

'O Apartamento é muitíssimo bem localizado, com todas as necessidades e comodidades para uma longa temporada, A Márcia e seu marido são excelentes anfitriões foram gentis e resolveram alguns imprevistos rapidamente e tranquilos. Minha família se sentiu em casa. Voltaremos sempre.'

'A estadia fói ótima. A casa possui boa localização, ótima estrutura e é exatamente como nas fotos. O anfitrião nos deixou um galão de agua e itens básicos para preparar comida e itens de higiene. Recomendamos.'

In [ ]:
# Removendo "<br>" e transformando os caracteres restantes em minúsculos
remove_br = lambda x: re.sub(r'\r<br/>', ' ', x.lower())
df.comments = df.comments.apply(remove_br)

print(df['comments'][315919],'\n')
print(df['comments'][237975])

o apartamento é muitíssimo bem localizado, com todas as necessidades e comodidades para uma longa temporada, a márcia e seu marido são excelentes anfitriões foram gentis e resolveram alguns imprevistos rapidamente e tranquilos. minha família se sentiu em casa. voltaremos sempre. 

a estadia fói ótima. a casa possui boa localização, ótima estrutura e é exatamente como nas fotos. o anfitrião nos deixou um galão de agua e itens básicos para preparar comida e itens de higiene. recomendamos.


In [ ]:
# Expandindo contrações
'''def expand_contractions(text, contraction_mapping=contractions_dict):
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), flags=re.IGNORECASE | re.DOTALL)

    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match) \
            if contraction_mapping.get(match) \
            else contraction_mapping.get(match.lower())
        expanded_contraction = first_char + expanded_contraction[1:]
        return expanded_contraction
  
    try:
      expanded_text = contractions_pattern.sub(expand_match, text)
      expanded_text = re.sub("'", "", expanded_text)
    except:
      return text
    return expanded_text

df.comments = df.comments.map(expand_contractions)

print(df['comments'][3],'\n')
print(df['comments'][20])'''

In [ ]:
# Removendo caracteres especiais
def remove_special_characters(text):
  text = re.sub(r'([{.(-)!}])', '', text)
  text = re.sub(r'[^ \\t\w\.]', '', text)
  text = re.sub(' +', ' ', text)
  return text

df.comments = df.comments.apply(remove_special_characters)

print(df['comments'][315919],'\n')
print(df['comments'][237975])

o apartamento é muitíssimo bem localizado com todas as necessidades e comodidades para uma longa temporada a márcia e seu marido são excelentes anfitriões foram gentis e resolveram alguns imprevistos rapidamente e tranquilos minha família se sentiu em casa voltaremos sempre 

a estadia fói ótima a casa possui boa localização ótima estrutura e é exatamente como nas fotos o anfitrião nos deixou um galão de agua e itens básicos para preparar comida e itens de higiene recomendamos


In [ ]:
# Traduzindo todas as reviews para o inglês
comments = df.comments.to_list()
comments_en = []
translator = Translator(service_urls=['translate.googleapis.com'])
translations = translator.translate(comments, dest='en')

for translation in translations:
    comments_en.append(translation.text)

df.insert(len(df.columns), 'comments_en', comments_en)

print(df['comments_en'][315919],'\n')
print(df['comments_en'][237975])

the apartment is very well located with all the necessities and amenities for a long stay Marcia and her husband are excellent hosts they were kind and resolved some unforeseen quickly and calmly my family felt at home we will always come back 

the stay was great the house has a good location great structure and it is exactly as in the photos the host left us a gallon of water and basic items to prepare food and hygiene items we recommend


In [ ]:
# Tokenizando
tk = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)

df.comments_en = df.comments_en.apply(tk.tokenize)

print(df['comments_en'][315919],'\n')
print(df['comments_en'][237975])

['the', 'apartment', 'is', 'very', 'well', 'located', 'with', 'all', 'the', 'necessities', 'and', 'amenities', 'for', 'a', 'long', 'stay', 'marcia', 'and', 'her', 'husband', 'are', 'excellent', 'hosts', 'they', 'were', 'kind', 'and', 'resolved', 'some', 'unforeseen', 'quickly', 'and', 'calmly', 'my', 'family', 'felt', 'at', 'home', 'we', 'will', 'always', 'come', 'back'] 

['the', 'stay', 'was', 'great', 'the', 'house', 'has', 'a', 'good', 'location', 'great', 'structure', 'and', 'it', 'is', 'exactly', 'as', 'in', 'the', 'photos', 'the', 'host', 'left', 'us', 'a', 'gallon', 'of', 'water', 'and', 'basic', 'items', 'to', 'prepare', 'food', 'and', 'hygiene', 'items', 'we', 'recommend']


In [ ]:
'''# Lemetizando
nlp = spacy.load('en')

def lemmatize(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

df.comments_en = df.comments_en.apply(lemmatize)

print(df['comments_en'][315919],'\n')
print(df['comments_en'][237975])'''

In [ ]:
# Removendo stop words
stopword_list = nltk.corpus.stopwords.words('english')
new_stopwords = ['rio','janeiro','copacabana','hide','airbnb','website','cancel']
stopword_list.extend(new_stopwords)

stop_lambda = lambda x: [y for y in x if y not in stopword_list]

df.comments_en = df.comments_en.apply(stop_lambda)

print(df['comments_en'][315919],'\n')
print(df['comments_en'][237975])

['apartment', 'well', 'located', 'necessities', 'amenities', 'long', 'stay', 'marcia', 'husband', 'excellent', 'hosts', 'kind', 'resolved', 'unforeseen', 'quickly', 'calmly', 'family', 'felt', 'home', 'always', 'come', 'back'] 

['stay', 'great', 'house', 'good', 'location', 'great', 'structure', 'exactly', 'photos', 'host', 'left', 'us', 'gallon', 'water', 'basic', 'items', 'prepare', 'food', 'hygiene', 'items', 'recommend']


In [ ]:
# Lemetizando
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(w) for w in (text)]

df.comments_en = df.comments_en.apply(lemmatize_text)

print(df['comments_en'][315919],'\n')
print(df['comments_en'][237975])

['apartment', 'well', 'located', 'necessity', 'amenity', 'long', 'stay', 'marcia', 'husband', 'excellent', 'host', 'kind', 'resolved', 'unforeseen', 'quickly', 'calmly', 'family', 'felt', 'home', 'always', 'come', 'back'] 

['stay', 'great', 'house', 'good', 'location', 'great', 'structure', 'exactly', 'photo', 'host', 'left', 'u', 'gallon', 'water', 'basic', 'item', 'prepare', 'food', 'hygiene', 'item', 'recommend']


In [ ]:
# Stemming
def stem_text(text):
    stemmer = PorterStemmer()
    return [stemmer.stem(w) for w in (text)]

df.comments_en = df.comments_en.apply(stem_text)

print(df['comments_en'][315919],'\n')
print(df['comments_en'][237975])

['apart', 'well', 'locat', 'necess', 'amen', 'long', 'stay', 'marcia', 'husband', 'excel', 'host', 'kind', 'resolv', 'unforeseen', 'quickli', 'calmli', 'famili', 'felt', 'home', 'alway', 'come', 'back'] 

['stay', 'great', 'hous', 'good', 'locat', 'great', 'structur', 'exactli', 'photo', 'host', 'left', 'u', 'gallon', 'water', 'basic', 'item', 'prepar', 'food', 'hygien', 'item', 'recommend']


In [ ]:
# Verificando valores nulos após a limpeza do texto
df.isna().sum()

listing_id       0
id               0
date             0
reviewer_id      0
reviewer_name    0
comments         0
comments_en      0
dtype: int64

In [ ]:
# Convertendo tokens em string
df.comments_en = df.comments_en.apply(' '.join)

In [ ]:
# Salvando o dataset após a limpeza do texto
df.to_csv('clean_reviews_en_30.csv', index=False, encoding='UTF-8') 